## Import Libraries and Globals

In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
from requests import get
import seaborn as sns
import os
import numpy as np
import random

# Read csv from data folder
part_summer_df = pd.read_csv("../data/raw/summer.csv")
part_winter_df = pd.read_csv("../data/raw/winter.csv")
dict_df = pd.read_csv("../data/raw/dictionary.csv")
codes_df = pd.read_csv("../data/raw/country_codes.csv")

ROOT_WIKI_BASE_URL = "https://en.wikipedia.org"
ROOT_WIKI_URL = "https://en.wikipedia.org/wiki/Lists_of_Olympic_medalists"

## Wrangle Data from Guardian

** Here is the link to data from 1896-2014: https://www.kaggle.com/the-guardian/olympic-games **

In [2]:
# Add seasonality
part_summer_df['Season'] = "Summer"
part_winter_df['Season'] = "Winter"

# merge the partials
merged_df = pd.concat([part_summer_df, part_winter_df], axis=0)

# Filter OUT mixed teams, special teams
# https://en.wikipedia.org/wiki/List_of_IOC_country_codes
filter_out_country_codes = ['ZZX','IOP', "EUN", "YUG", "BWI"] #  
merged_df = merged_df[~ merged_df['Country'].isin(filter_out_country_codes)]  
merged_df = merged_df[~merged_df['Country'].isnull()] 
merged_df.reset_index(inplace=True, drop = True)


In [3]:
# Change countries that no longer exist to the present day countries (This is not an exact science)
for index,row in merged_df.iterrows():
    ga3 = row['Country']
    if ga3 == "ANZ":
        merged_df.set_value(index, 'Country', "AUS")
    elif ga3 == "RU1":
        merged_df.set_value(index, 'Country', "RUS")
    elif ga3 == "TCH":
        rn = random.uniform(0,1)
        rc = "CZE" if rn < .5 else "SVK"
        merged_df.set_value(index, 'Country', rc)
    elif ga3 == "URS":
        merged_df.set_value(index, 'Country', "RUS")
    elif ga3 == "AHO":
        merged_df.set_value(index, 'Country', "NED")    
    
        
    

/anaconda3/envs/fight_prediction/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/fight_prediction/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/fight_prediction/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/fight_prediction/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/fight_prediction/lib/python3.6/site-packages/ipykernel/__main__.py:15: FutureWarning: set_value is deprecated and will be removed in a fut

## MERGE ON COUNTRY NAME

In [4]:
dict_df = dict_df.iloc[:,:2] # Subset dictionary df

In [5]:
temp_merged_df = pd.merge(merged_df, dict_df, left_on= "Country", right_on = "Code", how="left")
temp_merged_df = temp_merged_df.rename(columns = {'Country_x':'Guard_alpha_3', 'Country_y': 'Country'})
temp_merged_df = temp_merged_df.drop(['Code'], axis = 1)

In [6]:
# Wrangle country codes and regions
codes_ss_cols = ['name','alpha_3', 'region', 'sub_region']
codes_df_ss = codes_df[codes_ss_cols]

In [7]:
temp_merged_df.head(3)

,Year,City,Sport,Discipline,Athlete,Guard_alpha_3,Gender,Event,Medal,Season,Country
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Summer,Hungary
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver,Summer,Austria
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze,Summer,Greece


In [8]:
codes_df_ss.head(3)

,name,alpha_3,region,sub_region
0,Afghanistan,AFG,Asia,Southern Asia
1,Åland Islands,ALA,Europe,Northern Europe
2,Albania,ALB,Europe,Southern Europe


In [9]:
# merge the country codes and regions to the olympic data set
final_mm_df = pd.merge(temp_merged_df, codes_df_ss, left_on= "Country", right_on = "name", how="left")
final_mm_df = final_mm_df.reset_index()
final_mm_df['Country'] = final_mm_df['Country'].replace("*","")

In [10]:
final_mm_df = final_mm_df.drop(columns= "index")

In [11]:
final_mm_df.describe()

,Year
count,36150.000000
mean,1972.869544
std,32.590973
min,1896.000000
25%,1952.000000
50%,1980.000000
75%,2000.000000
max,2014.000000


In [12]:
final_mm_df.to_csv("../data/processed/merged_data.csv")

# Exploratory Data Analysis

## Scraping with BeautifulSoup

In [14]:
summer_2016_url = 'https://en.wikipedia.org/wiki/List_of_2016_Summer_Olympics_medal_winners'
winter_2018_url = 'https://en.wikipedia.org/wiki/List_of_2018_Winter_Olympics_medal_winners'

urls = [summer_2016_url, winter_2018_url]

In [17]:
d = {}
d['Year'] = []
d['City'] = []
d['Sport'] = []
d['Discipline'] = []
d['Athlete'] = []
d['Country'] = []
d['Gender'] = []
d['Event'] = []
d['Medal'] = []
d['Season'] = []

season_inx = 0

for url in urls:
    if(url == 0):
        year = 2016
        city = Rio de Janeiro
        season = "Summer"
    else:
        year = 2018
        city = "Pyeongchang"
        season = "Winter"
        
    page = get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    print(soup)

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of 2016 Summer Olympics medal winners - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_2016_Summer_Olympics_medal_winners","wgTitle":"List of 2016 Summer Olympics medal winners","wgCurRevisionId":881038605,"wgRevisionId":881038605,"wgArticleId":44675550,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category link is on Wikidata","Commons category link is on Wikidata using P373","2016 Summer Olympics","Lists of Summer Olympic medalists by year","Medalists at the 2016 Summer Olympics","Rio de Janeiro (city)-related lists"],"wgB

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of 2018 Winter Olympics medal winners - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_2018_Winter_Olympics_medal_winners","wgTitle":"List of 2018 Winter Olympics medal winners","wgCurRevisionId":880976639,"wgRevisionId":880976639,"wgArticleId":56336598,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from January 2018","2018 Winter Olympics","Lists of Winter Olympic medalists by year","South Korea sports-related lists","Medalists at the 2018 Winter Olympics"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageCont

## Test Environment for Scraping

In [28]:
t_page = get("https://en.wikipedia.org/wiki/List_of_2016_Summer_Olympics_medal_winners")
t_soup = BeautifulSoup(t_page.content, 'html.parser')